# Grid Cross Validation

In [1]:
from osgeo import gdal, gdal_array
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score
import numpy as np
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from pprint import pprint

gdal.UseExceptions()
gdal.AllRegister()

In [2]:
#read all training raster dataset
tr_stacked = gdal.Open('trainingImage/stacked_indicies_kampar.tif', gdal.GA_ReadOnly)
tr_smi = gdal.Open('trainingImage/smi_training_1.tif', gdal.GA_ReadOnly)

In [3]:
#convert to array for training data
stacked_zeros = np.zeros((tr_stacked.RasterYSize, tr_stacked.RasterXSize, tr_stacked.RasterCount),
                            gdal_array.GDALTypeCodeToNumericTypeCode(tr_stacked.GetRasterBand(1).DataType))
for a in range(stacked_zeros.shape[2]):
    stacked_zeros[:, :, a] = tr_stacked.GetRasterBand(a + 1).ReadAsArray()
smi = tr_smi.GetRasterBand(1).ReadAsArray().astype(np.float32)
# smi_masked = np.ma.masked_where(smi == 0, smi)

In [4]:
#let's create feature and label array of training data
x = stacked_zeros[smi > 0, :] #feature
y = smi[smi > 0] #label
print('X shape {x} and Y shape {y}'.format(x = x.shape, y = y.shape))

X shape (176810, 15) and Y shape (176810,)


In [5]:
#split our data to train and test data by 75% and 25%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
print('Training features shape :', x_train.shape)
print('Training labels shape :', y_train.shape)
print('Testing features shape', x_test.shape)
print('Testing labels shape:', y_test.shape)

Training features shape : (132607, 15)
Training labels shape : (132607,)
Testing features shape (44203, 15)
Testing labels shape: (44203,)


In [6]:
#Grid Search cross validation. The parameter based on random search cross validation
param_grid = {
    'bootstrap' : [True, False], #False
    'max_depth' : [42, None], #None
    'max_features' : ['auto', 'sqrt'], #auto
    'min_samples_leaf' : [1, 2], #1
    'min_samples_split' : [2, 3], #2
    'n_estimators' : [200] #200
}
pprint(param_grid)

#create a base model
rfReg = RandomForestRegressor(random_state = 42)
#initiate grid model
grid_search = GridSearchCV(estimator = rfReg, param_grid = param_grid, 
                           cv = 3, n_jobs = -1, verbose = 2, return_train_score = True)

{'bootstrap': [True, False],
 'max_depth': [42, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2],
 'min_samples_split': [2, 3],
 'n_estimators': [200]}


In [7]:
#fit the grid
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 132.9min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 243.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs

In [8]:
#print the best parameters
pprint(grid_search.best_params_)

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}


In [9]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    rsq = r2_score(test_labels, predictions)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('R Squared score = {r}'.format(r = rsq))
    
    return accuracy

In [10]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, x_test, y_test)

Model Performance
Average Error: 0.0792 degrees.
Accuracy = 83.36%.
R Squared score = 0.5526341224757939
